In [1]:
import pandas as pd
import pytz
from datetime import datetime

# Function to standardize timestamps with EST/EDT to UTC
def standardize_timestamp_v2(timestamp_str):
    try:
        # Extract timezone and date-time parts from the string
        time_zone_str = timestamp_str[-3:]
        date_time_str = timestamp_str[:-4]

        # Choose format based on presence of AM/PM
        if 'AM' in date_time_str or 'PM' in date_time_str:
            date_time_format = '%Y%m%d %I:%M %p'  # 12-hour format
        else:
            date_time_format = '%Y%m%d %H:%M'  # 24-hour format

        # Set timezone information
        if time_zone_str in ['EST', 'EDT']:
            time_zone = pytz.timezone('America/New_York')
        elif time_zone_str == 'UTC':
            time_zone = pytz.utc
        else:
            return timestamp_str  # Return original if timezone is unrecognized

        # Parse date-time string and localize it
        date_time_obj = datetime.strptime(date_time_str, date_time_format)
        localized_date_time = time_zone.localize(date_time_obj)

        # Convert to UTC
        utc_date_time = localized_date_time.astimezone(pytz.utc)
        return utc_date_time.strftime('%m/%d/%Y %H:%M')  # Format to standardized form
    except ValueError:
        return timestamp_str  # Return original on parsing error

# Function to clean and standardize timestamps
def clean_and_standardize_timestamp(timestamp_str):
    try:
        # Remove extraneous characters like 'Â'
        timestamp_str = timestamp_str.replace('Â', '')

        # Extract timezone and date-time parts
        time_zone_str = timestamp_str[-3:]
        date_time_str = timestamp_str[:-3].strip()

        # Choose format based on presence of AM/PM
        if 'AM' in date_time_str or 'PM' in date_time_str:
            date_time_format = '%Y%m%d %I:%M%p'  # 12-hour format
        else:
            date_time_format = '%Y%m%d %H:%M'  # 24-hour format

        # Set timezone information
        if time_zone_str in ['EST', 'EDT']:
            time_zone = pytz.timezone('America/New_York')
        elif time_zone_str == 'UTC':
            time_zone = pytz.utc
        else:
            return timestamp_str  # Return original if timezone is unrecognized

        # Parse and localize date-time string
        date_time_obj = datetime.strptime(date_time_str, date_time_format)
        localized_date_time = time_zone.localize(date_time_obj)

        # Convert to UTC
        utc_date_time = localized_date_time.astimezone(pytz.utc)
        return utc_date_time.strftime('%m/%d/%Y %H:%M')  # Standardize format
    except ValueError:
        return timestamp_str  # Return original on parsing error

# Load the DataFrame from TSV file
file_path = r"C:\Users\hassa\Downloads\Final_data.tsv"   
df = pd.read_csv(file_path, sep='\t')

# Apply the functions to standardize timestamps
df['ts_utc_cleaned'] = df['ts'].apply(standardize_timestamp_v2)
df['ts_utc_standardized'] = df['ts'].apply(clean_and_standardize_timestamp)

# Merge the standardized timestamp columns
def select_standardized_ts(row):
    # Select UTC standardized timestamp if original is UTC, else EST/EDT
    if "UTC" in row['ts']:
        return row['ts_utc_standardized']
    else:
        return row['ts_utc_cleaned']

# Apply the merge function
df['ts_standardized'] = df.apply(select_standardized_ts, axis=1)

# Save the final DataFrame to a new TSV file
output_file_path = r"C:\Users\hassa\OneDrive\Desktop\Test_data_files\final_timestamp_adjusted_1.tsv"  
df.to_csv(output_file_path, sep='\t', index=False)

print("File saved to:", output_file_path)


File saved to: C:\Users\hassa\OneDrive\Desktop\Test_data_files\final_timestamp_adjusted_1.tsv
